In [ ]:
from app.flirimageextractor import FlirImageExtractor
import os
import cv2
import numpy as np

os.chdir("C:/Users/Bunni/Documents/FinalProject/ThermoML")

FLIR thermal images creation

In [ ]:
import glob
from pathlib import Path

# Initialize FLIR extractor
flir = FlirImageExtractor()
folder_path = os.path.abspath('Data/images')
output_dir = os.path.join(folder_path, "magma")
os.makedirs(output_dir, exist_ok=True)

for file_path in glob.glob(os.path.join(folder_path, "*.jpg")):
    # Load the thermal image
    flir.process_image(file_path, True)
    file = Path(file_path)

    # Extract temperature matrix
    thermal_data = flir.get_thermal_np()
    thermal_gray = cv2.normalize(thermal_data, None, 0, 255, cv2.NORM_MINMAX)
    thermal_gray = thermal_gray.astype(np.uint8)

    # Apply a colormap directly
    thermal_color = cv2.applyColorMap(thermal_gray, cv2.COLORMAP_INFERNO)

    #cv2.imwrite('new_image.jpg', thermal_gray) # Save the image
    output_path = os.path.join(output_dir, f"{file.name}.jpg")
    cv2.imwrite(output_path, thermal_color)

    # Save temperature data as CSV
    np.savetxt("temperature_data.csv", thermal_data, delimiter=",")

    # Show the thermal image
    flir.save_images()

FLIR optical images extraction

In [ ]:
import subprocess
from pathlib import Path

folder_path = os.path.abspath('Data/images')

# In bash: $ exiftool -b -EmbeddedImage -42 Data/images/FLIR1231.jpg > optical_FLIR1231.jpg
for file_path in glob.glob(f"{folder_path}/*.jpg"):
    file = Path(file_path)
    optical_path = folder_path + "\\optical\\" + "optical_" + file.name
    subprocess.run(["exiftool", "-b", "-EmbeddedImage", "-42", file_path], stdout=open(optical_path, "wb"))

<>:9: SyntaxWarning: invalid escape sequence '\o'
<>:9: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Bunni\AppData\Local\Temp\ipykernel_16936\3076532605.py:9: SyntaxWarning: invalid escape sequence '\o'
  optical_path = folder_path + "\optical\\" + "optical_" + file.name


In [ ]:
folder_path = os.path.abspath('Data/images')

folder_path = folder_path + "/optical"

for file_path in glob.glob(f"{folder_path}/*.jpg"):
    optical_img_gray = cv2.imread(optical_path, cv2.IMREAD_GRAYSCALE)
    optical_img_color = cv2.imread(optical_path, cv2.IMREAD_ANYCOLOR)

In [ ]:
folder_path = os.path.abspath('Data/images')

folder_path = folder_path + "/bwr"

desired_size = (320, 240)  # (width, height)

print(folder_path)
# In bash: $ exiftool -b -EmbeddedImage -42 Data/images/FLIR1231.jpg > optical_FLIR1231.jpg
for file_path in glob.glob(f"{folder_path}/*.jpg"):
    thermal_img = cv2.imread(file_path, cv2.IMREAD_ANYCOLOR)
    
    # Resize only if needed
    if thermal_img.shape[1] != desired_size[0] or thermal_img.shape[0] != desired_size[1]:
        thermal_img = cv2.resize(thermal_img, desired_size, interpolation=cv2.INTER_AREA)
    
    if thermal_img is None:
       print("Error: Optical image extraction failed!")
    else:
        cv2.imshow("Optical Image", thermal_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [1]:
from segment_anything import sam_model_registry
import torch

sam = sam_model_registry["vit_h"](checkpoint="C:/Users/Bunni/Documents/FinalProject/ThermoML/models/sam_vit_h_4b8939.pth").to("cuda")

Below we separate thermal pictures that were extracted using the thermal data with the cv2 inferno color using the Meta SAM model. We get a segmentation of each hand separately.

We achieve successful(both hands) segmentation at 70%. This rate includes cold hands that struggle to be detected, excluding cold hands the rate is bigger around 90%.

In [ ]:
import os
import glob
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamPredictor

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)
    blended = (image * alpha).astype(np.uint8)
    return blended

# === Settings ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"
inferno_dir = os.path.join(project_dir, "data", "images", "inferno")
desired_size = (320, 240)
min_area = 1000

# === Load SAM ===
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "backend", "models", "sam_vit_h_4b8939.pth")
).to("cuda")
predictor = SamPredictor(sam)

# === Output directory ===
output_dir = os.path.join(project_dir, "inferno_output")
os.makedirs(output_dir, exist_ok=True)
print(f"\n Processing inferno images in: {inferno_dir}")

# === Process only inferno images ===
for file_path in glob.glob(f"{inferno_dir}/*.jpg"):
    filename = os.path.basename(file_path)
    print(f"\n Processing image: {filename}")

    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    if image.shape[1] != desired_size[0] or image.shape[0] != desired_size[1]:
        image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

    image = enhance_contrast(image)
    predictor.set_image(image)

    h, w = desired_size[1], desired_size[0]
    input_point = np.array([
        [int(w * 0.25), int(h * 0.6)],
        [int(w * 0.75), int(h * 0.6)],
        [int(w * 0.25), int(h * 0.4)],
        [int(w * 0.75), int(h * 0.4)],
        [int(w * 0.5), int(h * 0.85)]
    ])
    input_label = np.array([1, 1, 1, 1, 0])

    masks, _, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=False
    )

    mask = masks[0].astype(np.uint8)
    num_labels, labels = cv2.connectedComponents(mask)
    print(f" Found {num_labels - 1} components.")

    hands = []
    for label in range(1, num_labels):
        hand_mask = (labels == label).astype(np.uint8)
        area = cv2.countNonZero(hand_mask)
        if area < min_area:
            continue
        M = cv2.moments(hand_mask)
        if M["m00"] == 0:
            continue
        center_x = int(M["m10"] / M["m00"])
        hands.append((center_x, hand_mask))

    if len(hands) == 1:
        print("Only 1 component found attempting to split")
        big_mask = hands[0][1]
        mid_x = w // 2
        left_mask = np.zeros_like(big_mask)
        right_mask = np.zeros_like(big_mask)
        left_mask[:, :mid_x] = big_mask[:, :mid_x]
        right_mask[:, mid_x:] = big_mask[:, mid_x:]
        if cv2.countNonZero(left_mask) > min_area and cv2.countNonZero(right_mask) > min_area:
            hands = [(int(w * 0.25), left_mask), (int(w * 0.75), right_mask)]
        else:
            print("Failed to split  skipping image.")
            continue

    if len(hands) != 2:
        print("Skipping did not find exactly 2 hands.")
        continue

    hands.sort(key=lambda h: h[0])
    base_name = os.path.splitext(filename)[0]

    for i, (center_x, hand_mask) in enumerate(hands):
        side = "left" if i == 0 else "right"
        feathered_hand = feather_edges(image, hand_mask, feather_amount=5)
        output_path = os.path.join(output_dir, f"hand_{side}_{base_name}.jpg")
        cv2.imwrite(output_path, cv2.cvtColor(feathered_hand, cv2.COLOR_RGB2BGR))
        print(f" Saved {side} hand to: {output_path}")

Below we separate the segmented hands that we got from the SAM into the joints, and the tips of the fingers using the google HandLandMarker model.

We achieve successful joint recognition at 100% of all successful segmented hands.

In [ ]:
import os
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML/backend"
input_dir = os.path.join(project_dir, "optical_output")
output_dir = os.path.join(project_dir, "optical_landmarks")
os.makedirs(output_dir, exist_ok=True)

# === Load MediaPipe model ===
model_path = os.path.join(project_dir, "backend", "models", "hand_landmarker.task")
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

landmarker = vision.HandLandmarker.create_from_options(options)

# === Process each image ===
for file in os.listdir(input_dir):
    if not file.lower().endswith((".jpg", ".png", ".jpeg")):
        continue 

    image_path = os.path.join(input_dir, file)
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

    result = landmarker.detect(mp_image)

    if result.hand_landmarks:
        for hand in result.hand_landmarks:
            for landmark in hand:
                x_px = int(landmark.x * image.shape[1])
                y_px = int(landmark.y * image.shape[0])
                cv2.circle(image, (x_px, y_px), 3, (0, 255, 0), -1)

    output_path = os.path.join(output_dir, f"landmarks_{file}")
    cv2.imwrite(output_path, image)
    print(f"Saved landmark image to: {output_path}")

Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1231.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1232.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1233.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1234.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1235.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1236.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1237.jpg
Saved landmark image to: C:/Users/Bunni/Documents/FinalProject/ThermoML\inferno_landmarks_reversy\landmarks_FLIR1238.jpg
Saved landmark image to: C:/User

Sorting out data

In [ ]:
import pandas as pd
import os

project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML/"

# Load Excel files
df_images = pd.read_excel(os.path.join(project_dir, "Data", "TH-SRG01 FLIR numbers May2025.xlsx"))
df_labels = pd.read_excel(os.path.join(project_dir, "Data", "TH-SRG01 Investigator Database 16May25.xlsx"))

# Define the exact column order based on your description
label_columns = [
    "TENDERNESS_WRIST_RIGHT", "TENDERNESS_WRIST_LEFT", "TENDERNESS_CMC1_RIGHT", "TENDERNESS_CMC1_LEFT",
    "TENDERNESS_MCP1_RIGHT", "TENDERNESS_MCP1_LEFT", "TENDERNESS_IP1_RIGHT", "TENDERNESS_IP1_LEFT",
    "TENDERNESS_MCP2_RIGHT", "TENDERNESS_MCP2_LEFT", "TENDERNESS_PIP2_RIGHT", "TENDERNESS_PIP2_LEFT",
    "TENDERNESS_DIP2_RIGHT", "TENDERNESS_DIP2_LEFT", "TENDERNESS_MCP3_RIGHT", "TENDERNESS_MCP3_LEFT",
    "TENDERNESS_PIP3_RIGHT", "TENDERNESS_PIP3_LEFT", "TENDERNESS_DIP3_RIGHT", "TENDERNESS_DIP3_LEFT",
    "TENDERNESS_MCP4_RIGHT", "TENDERNESS_MCP4_LEFT", "TENDERNESS_PIP4_RIGHT", "TENDERNESS_PIP4_LEFT",
    "TENDERNESS_DIP4_RIGHT", "TENDERNESS_DIP4_LEFT", "TENDERNESS_MCP5_RIGHT", "TENDERNESS_MCP5_LEFT",
    "TENDERNESS_PIP5_RIGHT", "TENDERNESS_PIP5_LEFT", "TENDERNESS_DIP5_RIGHT", "TENDERNESS_DIP5_LEFT"
]

# Extract label columns in order, resulting in shape (num_rows, 32)
labels = df[label_columns].to_numpy()

# Access example:
# labels[0][0] is the value of TENDERNESS_WRIST_RIGHT in the first (sorted) row


# --- Rename 'Code' column to match ---
df_labels.rename(columns={"code": "Patient Code"}, inplace=True)

# --- Define tenderness columns to keep ---
target_tenderness_cols = [
    "TENDERNESS_WRIST_RIGHT", "TENDERNESS_WRIST_LEFT",
    "TENDERNESS_CMC1_RIGHT", "TENDERNESS_CMC1_LEFT",
    "TENDERNESS_MCP1_RIGHT", "TENDERNESS_MCP1_LEFT",
    "TENDERNESS_IP1_RIGHT", "TENDERNESS_IP1_LEFT",
    "TENDERNESS_MCP2_RIGHT", "TENDERNESS_MCP2_LEFT",
    "TENDERNESS_PIP2_RIGHT", "TENDERNESS_PIP2_LEFT",
    "TENDERNESS_DIP2_RIGHT", "TENDERNESS_DIP2_LEFT",
    "TENDERNESS_MCP3_RIGHT", "TENDERNESS_MCP3_LEFT",
    "TENDERNESS_PIP3_RIGHT", "TENDERNESS_PIP3_LEFT",
    "TENDERNESS_DIP3_RIGHT", "TENDERNESS_DIP3_LEFT",
    "TENDERNESS_MCP4_RIGHT", "TENDERNESS_MCP4_LEFT",
    "TENDERNESS_PIP4_RIGHT", "TENDERNESS_PIP4_LEFT",
    "TENDERNESS_DIP4_RIGHT", "TENDERNESS_DIP4_LEFT",
    "TENDERNESS_MCP5_RIGHT", "TENDERNESS_MCP5_LEFT",
    "TENDERNESS_PIP5_RIGHT", "TENDERNESS_PIP5_LEFT",
    "TENDERNESS_DIP5_RIGHT", "TENDERNESS_DIP5_LEFT"
]

df_labels[target_tenderness_cols] = df_labels[target_tenderness_cols].replace(".", 0).fillna(0).astype(int)

# --- Enhance tenderness labels using OR logic ---
# CMC1 (Right and Left)
df_labels["TENDERNESS_CMC1_RIGHT"] = (
    (df_labels["CMC1_GRIND_RIGHT"] == 1) | (df_labels["TENDERNESS_CMC1_RIGHT"] == 1)
).astype(int)

df_labels["TENDERNESS_CMC1_LEFT"] = (
    (df_labels["CMC1_GRIND_LEFT"] == 1) | (df_labels["TENDERNESS_CMC1_LEFT"] == 1)
).astype(int)

# MCP1 (Right and Left)
df_labels["TENDERNESS_MCP1_RIGHT"] = (
    (df_labels["MCP1_INSTABILITY_RIGHT"] == 1) | (df_labels["TENDERNESS_MCP1_RIGHT"] == 1)
).astype(int)

df_labels["TENDERNESS_MCP1_LEFT"] = (
    (df_labels["MCP1_INSTABILITY_LEFT"] == 1) | (df_labels["TENDERNESS_MCP1_LEFT"] == 1)
).astype(int)

# Wrist (Right and Left)
df_labels["TENDERNESS_WRIST_RIGHT"] = (
    (df_labels["TENDERNESS_WRIST_RADIAL_RIGHT"] == 1) |
    (df_labels["TENDERNESS_WRIST_ULNAR_RIGHT"] == 1) |
    (df_labels["סמן אם יש: | FINKLESTEIN TEST: DQ | יד ימין"] == 1) |
    (df_labels["סמן אם יש: | ECU TENDERNESS | יד ימין"] == 1)
).astype(int)

df_labels["TENDERNESS_WRIST_LEFT"] = (
    (df_labels["TENDERNESS_WRIST_RADIAL_LEFT"] == 1) |
    (df_labels["TENDERNESS_WRIST_ULNAR_LEFT"] == 1) |
    (df_labels["סמן אם יש: | FINKLESTEIN TEST: DQ | יד שמאל"] == 1) |
    (df_labels["סמן אם יש: | ECU TENDERNESS | יד שמאל"] == 1)
).astype(int)

# --- Filter columns to keep ---
df_labels_filtered = df_labels[["Patient Code"] + target_tenderness_cols]

# --- Merge ---
df_merged = pd.merge(df_images, df_labels_filtered, on="Patient Code", how="inner")

# --- Save ---
output_path = os.path.join(project_dir, "Data", "merged_inflammation_dataset.csv")
df_merged.to_csv(output_path, index=False)
print("Saved to:", output_path)

Filter out rows that are not in image database

In [ ]:
import pandas as pd
import numpy as np
import os
import re

# Load CSV
df = pd.read_csv('your_file.csv')

# Folder with your actual images
image_folder = 'path_to_images'

available_flir_ids = set()

for filename in os.listdir(image_folder):
    if filename.startswith('masked_FLIR') and filename.endswith(('.jpg', '.png')):
        match = re.search(r'masked_FLIR(\d+)', filename)
        if match:
            flir_id = int(match.group(1))
            available_flir_ids.add(flir_id)

# Filter DataFrame to only include rows with FLIR IDs present in the image folder
df = df[df['FLIR'].isin(available_flir_ids)].sort_values(by='FLIR').reset_index(drop=True)

Ordering landmarks

In [ ]:
import numpy as np

def reorder_landmarks_if_needed(landmarks):
    """
    Ensures that landmarks of the left hand (with smaller x) come before the right hand.
    Assumes:
    - landmarks is a (42, 2) or (42, 3) array.
    - landmarks[0:21] are one hand, landmarks[21:42] are the other.
    """

    # Check if we have exactly 42 landmarks (2 hands)
    if landmarks.shape[0] != 42:
        return landmarks  # No change needed

    left_hand = landmarks[:21]
    right_hand = landmarks[21:]

    # Compare x-coordinates (column 0) of landmark 0 and 21
    if landmarks[0][0] > landmarks[21][0]:  # Left hand is actually second in the array
        landmarks = np.concatenate([right_hand, left_hand], axis=0)

    return landmarks

def remap_landmarks(landmarks):
    """
    Reorders landmarks according to a custom label mapping and removes irrelevant points.

    Args:
        landmarks (np.ndarray): Original array of shape (42, N) where N=2 or 3 (x, y, [z])

    Returns:
        np.ndarray: Remapped landmark array of shape (32, N)
    """
    # Irrelevant landmarks to remove
    irrelevant_indices = {4, 8, 12, 16, 20, 25, 29, 33, 37, 41}

    # Mapping: new_index -> old_index
    mapping = [
        21, 0, 22, 1, 23, 2, 24, 3,
        26, 5, 27, 6, 28, 7, 30, 9,
        31, 10, 32, 11, 34, 13, 35, 14,
        36, 15, 38, 17, 39, 18, 40, 19
    ]

    # Apply mapping and ignore irrelevant indices
    filtered_landmarks = np.array([landmarks[i] for i in mapping if i not in irrelevant_indices])

    return filtered_landmarks

all_landmarks = np.load(os.path.join(project_dir, "landmarks_optical", "all_optical_landmarks.npy"))
reordered_landmarks = np.array([
    reorder_landmarks_if_needed(landmarks)
    for landmarks in all_landmarks
])

# Optional: Save or use reordered_landmarks here
# np.save(os.path.join(project_dir, "reordered_landmarks.npy"), reordered_landmarks)

# === Step 2: Remap after reorder ===
remapped_landmarks = np.array([
    remap_landmarks(landmarks)
    for landmarks in reordered_landmarks
])

In [ ]:
import cv2
import numpy as np
import torch

def prepare_4_channel_image(thermal_image, segmentation_mask, hand_centers=None):
    if thermal_image.ndim == 3 and thermal_image.shape[2] == 3:
        pil_gray = Image.fromarray(thermal_image).convert("L")
        thermal_image = np.array(pil_gray).astype(np.float32)
    elif thermal_image.ndim == 2:
        thermal_image = thermal_image.astype(np.float32)
    else:
        raise ValueError("Expected RGB or 2D image")

    # Normalize
    thermal = (thermal_image - thermal_image.min()) / (thermal_image.max() - thermal_image.min() + 1e-6)
    thermal = torch.tensor(thermal, dtype=torch.float32)
    mask = torch.tensor(segmentation_mask, dtype=torch.float32)
    masked_thermal = thermal * mask
    H, W = thermal.shape

    if hand_centers is not None and len(hand_centers) > 0:
        y_grid, x_grid = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
        distances = [torch.sqrt((x_grid - cx)**2 + (y_grid - cy)**2) for cx, cy in hand_centers]
        distance_map = torch.min(torch.stack(distances, dim=0), dim=0).values
        distance_map = distance_map / (distance_map.max() + 1e-6)
    else:
        distance_map = torch.nn.functional.avg_pool2d(mask.unsqueeze(0).unsqueeze(0), kernel_size=11, stride=1, padding=5).squeeze()

    image_4ch = torch.stack([thermal, mask, masked_thermal, distance_map], dim=0)
    return image_4ch

# No need for this function but it's kept here 
def clean_mask(mask: np.ndarray, kernel_size=5):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  # removes small noise
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)  # fills small holes
    return cleaned

def get_hand_centers(landmarks):
    centers = []
    centers.append(intersect(landmarks[1], landmarks[17], landmarks[12], landmarks[0]))
    centers.append(intersect(landmarks[22], landmarks[38], landmarks[33], landmarks[21]))

def intersect(p1, p2, p3, p4):
    a1 = p2[1] - p1[1]
    b1 = p1[0] - p2[0]
    c1 = a1 * p1[0] + b1 * p1[1]

    a2 = p4[1] - p3[1]
    b2 = p3[0] - p4[0]
    c2 = a2 * p3[0] + b2 * p3[1]

    determinant = a1 * b2 - a2 * b1
    if determinant == 0:
        return None
    x = (b2 * c1 - b1 * c2) / determinant
    y = (a1 * c2 - a2 * c1) / determinant
    return int(x), int(y)

# Transform mask image into binary mask for second channel
# In actual use, using 1/8 of the max value in the image or lower is working well
def load_segmented_image_as_mask(path, threshold=128):
    # Load as grayscale (0–255)
    seg_img = Image.open(path).convert("L")
    seg_array = np.array(seg_img)

    # Convert to binary mask: hand = 1, background = 0
    binary_mask = (seg_array > threshold).astype(np.uint8)
    return binary_mask  # Shape: (H, W), values 0 or 1

import os

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML/"
input_dir = os.path.join(project_dir, "backend", "inferno_landmarks_combined")
output_dir = os.path.join(project_dir, "backend", "inferno_landmarks_morph")
os.makedirs(output_dir, exist_ok=True)

# === Process each image ===
for file in os.listdir(input_dir):
    if not file.lower().endswith((".jpg", ".png", ".jpeg")):
        continue 

    image_path = os.path.join(input_dir, file)
    image = cv2.imread(image_path)
    
    #cleaned_image = clean_mask(image)
    binary_image = load_segmented_image_as_mask(image_path, np.max(image) // 8)

    output_path = os.path.join(output_dir, f"clean_{file}")
    cv2.imwrite(output_path, binary_image)
    print(f"Saved landmark image to: {output_path}")


Type: <class 'numpy.ndarray'>
Shape: (4, 240, 320)
Dtype: float32

thermal channel:
  Shape: (240, 320)
  Dtype: float32
  Unique values: [0.         0.00414938 0.00829876 0.01244813 0.01659751 0.02074689
 0.02489627 0.02904564 0.03319502 0.0373444  0.04149378 0.04564315
 0.04979253 0.05394191 0.05809129 0.06224066 0.06639004 0.07053942
 0.0746888  0.07883818 0.08298755 0.08713693 0.09128631 0.09543569
 0.09958506 0.10373444 0.10788382 0.1120332  0.11618257 0.12033195
 0.12448133 0.12863071 0.13278009 0.13692947 0.14107884 0.14522822
 0.1493776  0.15352698 0.15767635 0.16182573 0.16597511 0.17012449
 0.17427386 0.17842324 0.18257262 0.186722   0.19087137 0.19502075
 0.19917013 0.2033195  0.20746888 0.21161826 0.21576764 0.21991701
 0.22406639 0.22821577 0.23236515 0.23651452 0.2406639  0.24481328
 0.24896266 0.25311205 0.25726143 0.2614108  0.26556018 0.26970956
 0.27385893 0.2780083  0.2821577  0.28630707 0.29045644 0.29460582
 0.2987552  0.30290458 0.30705395 0.31120333 0.3153527  0.

Landmarks JSON

In [ ]:
import os
import json
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"
input_dir = os.path.join(project_dir, "Data", "images", "inferno")
json_output_dir = os.path.join(project_dir, "inferno_jsons")
os.makedirs(json_output_dir, exist_ok=True)

# === Load MediaPipe model ===
model_path = os.path.join(project_dir, "backend", "models", "hand_landmarker.task")
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)
landmarker = vision.HandLandmarker.create_from_options(options)

# === Process each image ===
for file in os.listdir(input_dir):
    if not file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image_path = os.path.join(input_dir, file)
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

    result = landmarker.detect(mp_image)

    if result.hand_landmarks:
        hands_data = []
        for hand_landmarks in result.hand_landmarks:
            points = [{"x": float(l.x), "y": float(l.y), "z": float(l.z)} for l in hand_landmarks]
            avg_x = sum(p["x"] for p in points) / len(points)
            hands_data.append((avg_x, points))

        # Sort hands by x (from left to right on the image)
        hands_data.sort(key=lambda h: h[0])

        # Assign correct hand labels (left hand appears on left of image)
        labels = ["left", "right"]

        base_name = os.path.splitext(file)[0]

        for i, (_, landmarks) in enumerate(hands_data):
            json_filename = f"{base_name}_{labels[i]}_hand.json"
            json_path = os.path.join(json_output_dir, json_filename)

            with open(json_path, "w") as f:
                json.dump(landmarks, f, indent=4)

            print(f"✅ Saved {labels[i]} hand JSON to: {json_path}")

Landmarks model that checks how many landmarks on each hand and saves accordingly

In [ ]:
import os
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"
input_dir = os.path.join(project_dir, "optical_output", "inferno_output")
output_dir = os.path.join(project_dir, "backend", "optical_landmarks_combined")
os.makedirs(output_dir, exist_ok=True)

# === Load MediaPipe model ===
model_path = os.path.join(project_dir, "backend", "models", "hand_landmarker.task")
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

landmarker = vision.HandLandmarker.create_from_options(options)

# === Helper function to mark landmarks ===
def draw_landmarks_on_image(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
    result = landmarker.detect(mp_image)

    if result.hand_landmarks:
        for hand in result.hand_landmarks:
            for landmark in hand:
                x_px = int(landmark.x * image.shape[1])
                y_px = int(landmark.y * image.shape[0])
                cv2.circle(image, (x_px, y_px), 3, (0, 255, 0), -1)
        return image, sum(len(hand) for hand in result.hand_landmarks)
    return image, 0

# === Process segmented images ===
segmented_files = [f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

# Group left/right hand pairs
base_names = set()
for file in segmented_files:
    if "hand_left" in file or "hand_right" in file:
        base = file.replace("hand_left_", "").replace("hand_right_", "")
        base_names.add(base)
    else:
        base_names.add(file)

for base in base_names:
    left_path = os.path.join(input_dir, f"hand_left_{base}")
    right_path = os.path.join(input_dir, f"hand_right_{base}")
    both_path = os.path.join(input_dir, base)

    output_path = os.path.join(output_dir, f"landmarks_{base}")
    
    # Case 2: Process left + right if both exist
    has_left = os.path.exists(left_path)
    has_right = os.path.exists(right_path)
    has_base = os.path.exists(both_path)

    if has_left and has_right:
        left_img = cv2.imread(left_path)
        right_img = cv2.imread(right_path)

        left_marked, left_points = draw_landmarks_on_image(left_img)
        right_marked, right_points = draw_landmarks_on_image(right_img)

        combined = cv2.add(left_marked, right_marked)

        if left_points == 21 and right_points == 21:
            # Resize to match height
            # height = max(left_marked.shape[0], right_marked.shape[0])
            # left_resized = cv2.resize(left_marked, (left_marked.shape[1], height))
            # right_resized = cv2.resize(right_marked, (right_marked.shape[1], height))

            cv2.imwrite(output_path, combined)
            print(f"Saved COMBINED hands image: {output_path}")
        elif left_points == 42:
            cv2.imwrite(output_path, left_marked)
            print(f"Saved both hands through LEFT-only landmark image: {output_path}")
        elif right_points == 42:
            cv2.imwrite(output_path, right_marked)
            print(f"Saved both hands through RIGHT-only landmark image: {output_path}")
        else:
            print(f"Skipped {base} due to incomplete landmarks (left: {left_points}, right: {right_points})")
    elif has_left:
        left_img = cv2.imread(left_path)
        left_marked, left_points = draw_landmarks_on_image(left_img)
        if left_points == 42:
            cv2.imwrite(output_path, left_marked)
            print(f"Saved both hands through LEFT-only landmark image: {output_path}")
        else:
            print(f"Skipped {base} due to invalid LEFT-only landmarks ({left_points})")
    elif has_right:
        right_img = cv2.imread(right_path)
        right_marked, right_points = draw_landmarks_on_image(right_img)
        if right_points == 42:
            cv2.imwrite(output_path, right_marked)
            print(f"Saved both hands through RIGHT-only landmark image: {output_path}")
        else:
            print(f"Skipped {base} due to invalid RIGHT-only landmarks ({right_points})")
    elif has_base:
        combined_img = cv2.imread(both_path)
        both_marked, both_points = draw_landmarks_on_image(combined_img)
        if both_points == 42:
            cv2.imwrite(output_path, both_marked)
            print(f"Saved both hands through RIGHT-only landmark image: {output_path}")
        else:
            print(f"Skipped {base} due to invalid RIGHT-only landmarks ({both_points})")

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.metrics import confusion_matrix
import torch.nn as nn
import numpy as np
import os

# Define single joint classifier
class SingleJointInflammationClassifier(nn.Module):
    def __init__(self, num_landmarks=16):
        super().__init__()
        self.cnn = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.cnn.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.cnn.fc = nn.Identity()

        self.mlp = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 64)
        )

        self.classifier = nn.Sequential(
            nn.Linear(2048 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, image_input, landmark_input):
        x_img = self.cnn(image_input)
        x_landmark = self.mlp(landmark_input)
        x = torch.cat([x_img, x_landmark], dim=1)
        return self.classifier(x).squeeze(1)


# Define dataset for single joint
class SingleJointDataset(Dataset):
    def __init__(self, image_paths, landmark_data, joint_labels, transform=None):
        self.image_paths = image_paths
        self.landmark_data = landmark_data
        self.joint_labels = joint_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = torch.from_numpy(np.load(self.image_paths[idx])).float()
        # Normalize per channel
        for c in range(image.shape[0]):
            min_val = image[c].min()
            max_val = image[c].max()
            image[c] = (image[c] - min_val) / (max_val - min_val + 1e-6)
        if self.transform:
            image = self.transform(image)
        landmarks = torch.tensor(self.landmark_data[idx], dtype=torch.float32)
        label = torch.tensor(self.joint_labels[idx], dtype=torch.float32)
        return image, landmarks, label


# Training function
def train_single_joint(model, dataloader, optimizer, criterion, device, log_fn=print):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_preds = []

    for images, landmarks, labels in dataloader:
        images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, landmarks)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()

    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1])
    log_fn("Confusion Matrix:\n" + str(cm))
    return running_loss / len(dataloader)


# Evaluation function
def evaluate_single_joint(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, landmarks, labels in dataloader:
            images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
            outputs = model(images, landmarks)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"
image_dir = os.path.join(project_dir, "Data", "4_channel_images")
image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(".npy")]

processed_landmarks = np.load(os.path.join(project_dir, "landmarks_optical", "processed_optical_landmarks"))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# labels: shape (365, 32)
num_pos = labels.sum(axis=0)  # shape (32,)
num_neg = labels.shape[0] - num_pos  # shape (32,)

# Avoid division by zero
pos_weight = num_neg / (num_pos + 1e-5)

# Convert to PyTorch tensor
pos_weight_tensor = torch.tensor(pos_weight, dtype=torch.float32)

model_dir = os.path.join(project_dir, "trained_models")
log_dir = os.path.join(project_dir, "training_logs")
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

for joint_idx in range(32):
    model_path = os.path.join(model_dir, f"joint_{joint_idx}_resnet50.pt")
    log_path = os.path.join(log_dir, f"joint_{joint_idx}_training_log.txt")

    if os.path.exists(model_path):
        print(f"Model for joint {joint_idx} already exists. Skipping training.")
        continue

    with open(log_path, "w") as log_file:
        def log(s):
            print(s)
            log_file.write(s + "\n")

        log(f"\n=== Training model for Joint {joint_idx} ===")

        # Prepare data
        joint_landmarks = [lm[joint_idx] for lm in processed_landmarks]
        joint_labels = [label[joint_idx] for label in labels]
        dataset = SingleJointDataset(image_paths, joint_landmarks, joint_labels)

        train_size = int(0.7 * len(dataset))
        val_size = int(0.2 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

        model = SingleJointInflammationClassifier().to(device)
        train_labels = [label.item() for _, _, label in train_dataset]
        pos = sum(train_labels)
        neg = len(train_labels) - pos
        pos_weight = neg / (pos + 1e-5)
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight, dtype=torch.float32).to(device))
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

        best_val_loss = float('inf')
        best_model_state = None

        for epoch in range(60):
            train_loss = train_single_joint(model, train_loader, optimizer, criterion, device)
            val_loss, val_acc = evaluate_single_joint(model, val_loader, criterion, device)
            log(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.2f}")
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict()

        model.load_state_dict(best_model_state)

        test_loss, test_acc = evaluate_single_joint(model, test_loader, criterion, device)
        log(f"Joint {joint_idx} - Test Loss={test_loss:.4f}, Test Accuracy={test_acc:.2f}")
        log(f"Labels range: {np.min(joint_labels)}, {np.max(joint_labels)}")

        torch.save(model.state_dict(), model_path)
        log(f"Model saved to {model_path}")

Register optical to thermal using landmarks and scaling that was tested to make sure it is consistent throughout 

In [ ]:
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import matplotlib.pyplot as plt

def extract_hand_from_thermal(thermal_image, optical_segmented_image):
    # Create a binary mask where the optical image is not black
    # (any channel > 0 implies it's a hand pixel)
    hand_mask = np.any(optical_segmented_image != 0, axis=-1).astype(np.uint8)  # shape: (H, W)

    # Expand mask to 3 channels to match thermal image shape
    #hand_mask_3ch = np.repeat(hand_mask[:, :, np.newaxis], 3, axis=2)  # shape: (H, W, 3)

    # Apply mask to thermal image
    masked_thermal = cv2.bitwise_and(thermal_image, thermal_image, mask=hand_mask)

    return masked_thermal

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"

# === Load MediaPipe model ===
model_path = os.path.join(project_dir, "backend", "models", "hand_landmarker.task")
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

landmarker = vision.HandLandmarker.create_from_options(options)

def detect_and_draw_landmarks(image, landmarker):
    """
    Detect hand landmarks in an image and draw them on the image.

    Args:
        image (np.array): BGR image.
        landmarker: Initialized MediaPipe HandLandmarker.

    Returns:
        image_with_landmarks (np.array): RGB image with landmarks drawn.
        landmarks_list (list of tuples): List of (x, y) pixel coordinates for all detected landmarks.
        total_landmarks (int): Total number of landmarks detected.
    """
    # Convert BGR to RGB for MediaPipe processing
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
    result = landmarker.detect(mp_image)

    landmarks_list = []
    image_with_landmarks = rgb_image.copy()

    if result.hand_landmarks:
        for hand_landmarks in result.hand_landmarks:
            for landmark in hand_landmarks:
                x_px = int(landmark.x * image.shape[1])
                y_px = int(landmark.y * image.shape[0])
                landmarks_list.append((x_px, y_px))
                cv2.circle(image_with_landmarks, (x_px, y_px), 3, (0, 255, 0), -1)

    # Convert back to BGR before returning to keep consistent with OpenCV usage
    image_with_landmarks = cv2.cvtColor(image_with_landmarks, cv2.COLOR_RGB2BGR)
    return image_with_landmarks, landmarks_list, len(landmarks_list)

def func_landmark(left_img=None, right_img=None, combined_img=None, 
                  has_left=False, has_right=False, has_both=False, landmarker=landmarker_two_hand):
    """
    Process images to detect hand landmarks according to provided flags.

    Returns landmarks list if valid, else None.
    """

    if has_left and has_right:
        left_marked, left_landmarks, left_count = detect_and_draw_landmarks(left_img, landmarker)
        right_marked, right_landmarks, right_count = detect_and_draw_landmarks(right_img, landmarker)

        combined = cv2.add(left_img, right_img)
        combined_marked, combined_landmarks, combined_count = detect_and_draw_landmarks(combined, landmarker)

        if left_count == 21 and right_count == 21 and combined_count == 42:
            return combined_landmarks
        elif combined_count != 42:
            print("Error: Combined landmarks count not equal 42")
            return None
        elif left_count == 42:
            return left_landmarks
        elif right_count == 42:
            return right_landmarks
        else:
            print(f"Skipped: incomplete landmarks (left: {left_count}, right: {right_count})")
            return None

    elif has_left and left_img is not None:
        marked, landmarks, count = detect_and_draw_landmarks(left_img, landmarker)
        if count in [21, 42]:
            return landmarks
        print(f"Skipped: invalid left-only landmarks count: {count}")
        return None

    elif has_right and right_img is not None:
        marked, landmarks, count = detect_and_draw_landmarks(right_img, landmarker)
        if count == 42:
            return landmarks
        print(f"Skipped: invalid right-only landmarks count: {count}")
        return None

    elif has_both and combined_img is not None:
        marked, landmarks, count = detect_and_draw_landmarks(combined_img, landmarker)
        if count == 42:
            return landmarks
        print(f"Skipped: invalid both-only landmarks count: {count}")
        return None

    return None

def resize_scale(optical_image, scale_factor):
    """Resize optical image based on scale factor."""
    h, w = optical_image.shape[:2]
    new_w, new_h = int(w * scale_factor), int(h * scale_factor)
    return cv2.resize(optical_image, (new_w, new_h))

def center_pad_image(smaller_img, target_shape):
    small_h, small_w = smaller_img.shape[:2]
    target_h, target_w = target_shape[:2]
    
    top = (target_h - small_h) // 2
    bottom = target_h - small_h - top
    left = (target_w - small_w) // 2
    right = target_w - small_w - left

    padded_img = cv2.copyMakeBorder(smaller_img, top, bottom, left, right,
                                     borderType=cv2.BORDER_CONSTANT, value=0)
    return padded_img

def translate_image(image, x_shift=0, y_shift=0):
    """Translate image by x and y pixels. Positive y_shift moves down, negative moves up."""
    height, width = image.shape[:2]
    M = np.float32([[1, 0, x_shift], [0, 1, y_shift]])
    translated = cv2.warpAffine(image, M, (width, height), borderValue=(0,0,0))
    return translated

# === Paths ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
optical_folder = os.path.join(project_dir, "optical_output_feathered_blur")
thermal_folder = os.path.join(project_dir, "Data", "images", "inferno")
thermal_folder_landmark = os.path.join(project_dir, "inferno_output")
output_folder = os.path.join(project_dir, "masked_moprh_thermal_output_dynamic")
output_folder_shifted = os.path.join(project_dir, "Data", "images","shifted_inferno")
landmarks_output = os.path.join(project_dir, "landmarks_optical", "all_optical_landmarks.npy")
os.makedirs(output_folder, exist_ok=True)

landmarks = []

# === Loop through optical images and match with thermal ===
for optical_name in os.listdir(optical_folder):
    if not optical_name.endswith(".jpg"):
        continue

    # Extract base name: "hands_optical_FLIR1231.jpg" → "FLIR1231.jpg"
    if not optical_name.startswith("hands_optical_"):
        continue
    base_name = optical_name.replace("hands_optical_", "")
    segment_name_left = "hand_left_" + base_name
    segment_name_right = "hand_right_" + base_name
    morph_name = "clean_" + optical_name

    thermal_path = os.path.join(thermal_folder, base_name)
    thermal_segment_path_left = os.path.join(thermal_folder_landmark, segment_name_left)
    thermal_segment_path_right = os.path.join(thermal_folder_landmark, segment_name_right)
    optical_path = os.path.join(optical_folder, optical_name)

    # If we don't have left we also don't have right because of the way they are saved
    if not os.path.exists(thermal_path) or not os.path.exists(thermal_segment_path_left):
        print(f"Skipping {base_name}: thermal image not found.")
        continue

    # Load images
    thermal_image = cv2.imread(thermal_path)
    thermal_segment_image_left = cv2.imread(thermal_segment_path_left)
    thermal_segment_image_right = cv2.imread(thermal_segment_path_right)
    optical_segmented = cv2.imread(optical_path)

    scale = 1.41
    # Apply resizing
    optical_scaled = resize_scale(optical_segmented, scale)
    #optical_scaled = cv2.cvtColor(optical_scaled, cv2.COLOR_BGR2RGB)

    if thermal_image.shape[:2] != (240, 320):
        thermal_image = cv2.resize(thermal_image, (320, 240), interpolation=cv2.INTER_AREA)
        thermal_segment_image_left = cv2.resize(thermal_segment_image_left, (320, 240), interpolation=cv2.INTER_AREA)
        thermal_segment_image_right = cv2.resize(thermal_segment_image_right, (320, 240), interpolation=cv2.INTER_AREA)
    #thermal_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2RGB)
    thermal_padded = center_pad_image(thermal_image, optical_scaled.shape)
    thermal_segment_padded_left = center_pad_image(thermal_segment_image_left, optical_scaled.shape)
    thermal_segment_padded_right = center_pad_image(thermal_segment_image_right, optical_scaled.shape)
    
    thermal_padded = cv2.resize(thermal_padded, (320, 240), interpolation=cv2.INTER_AREA)
    thermal_segment_padded_left = cv2.resize(thermal_segment_padded_left, (320, 240), interpolation=cv2.INTER_AREA)
    thermal_segment_padded_right = cv2.resize(thermal_segment_padded_right, (320, 240), interpolation=cv2.INTER_AREA)
    optical_scaled = cv2.resize(optical_scaled, (320, 240), interpolation=cv2.INTER_AREA)

    thermal_landmarks_left = func_landmark(thermal_segment_padded_left, "", "", True, False, False)
    thermal_landmarks_right = func_landmark("", thermal_landmarks_right, "", True, False, False)
    optical_landmarks = func_landmark("", "", optical_scaled, False, False, True)
    # No optical or thermal landmarks detected, skip
    if ((thermal_landmarks_left == None and thermal_landmarks_right == None) or optical_landmarks == None):
        continue
    optical_point_left = 9
    optical_point_right = 30
    if (optical_landmarks[9][0] > (optical_scaled.shape[1] // 2)):
        optical_point_left = optical_point_left + 21
        optical_point_right = 9
    # No left landmarks, use the right
    if thermal_landmarks_left == None:
        x_shift = optical_landmarks[optical_point_right][0] - thermal_landmarks_right[9][0]
        y_shift = optical_landmarks[optical_point_right][1] - thermal_landmarks_right[9][1]
    # Use left landmarks
    else :
        x_shift = optical_landmarks[optical_point_left][0] - thermal_landmarks_left[9][0]
        y_shift = optical_landmarks[optical_point_left][1] - thermal_landmarks_left[9][1]

    shifted_thermal = translate_image(thermal_padded, y_shift=y_shift, x_shift=x_shift)

    # Apply mask
    masked_thermal = extract_hand_from_thermal(shifted_thermal, optical_scaled)

    # Save
    output_path = os.path.join(output_folder, f"masked_{base_name}")
    cv2.imwrite(output_path, masked_thermal)
    cv2.imwrite(output_folder_shifted, shifted_thermal)
    landmarks.append(optical_landmarks)
# Landmarks array save    
landmarks_np = np.array(landmarks)
np.save(landmarks_output, landmarks_np)

In [ ]:
import os
import json
import cv2
import numpy as np

# === Paths ===
project_dir = "C:/Users/Bunni/Documents/FinalProject/ThermoML"
landmark_dir = os.path.join(project_dir, "inferno_landmarks")

# === Load MediaPipe model ===
model_path = os.path.join(project_dir, "models", "hand_landmarker.task")

# List to collect filenames of images with a full hand detection
valid_images = []

# Iterate over all files in the directory
for filename in os.listdir(landmark_dir):
    # Only process files that start with "landmarks_" (to match the expected files)
    if not filename.startswith("landmarks_"):
        continue
    file_path = os.path.join(landmark_dir, filename)

    # Attempt to load landmark coordinates from a corresponding JSON file (if it exists)
    coords = None
    base_name, ext = os.path.splitext(file_path)
    json_path = base_name + ".json"  # e.g., "inferno_landmarks/landmarks_FLIR1231_left_hand.json"
    if os.path.exists(json_path):
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = None

        if data is not None:
            # Determine how the coordinates are stored and extract them
            if isinstance(data, list):
                # If data is a list, it could be:
                # - A list of 21 coordinate points (each point might be [x,y] or a dict with coords).
                # - A list of multiple hand landmark sets (e.g., [hand1_points, hand2_points]).
                if len(data) == 21:
                    coords = data  # 21 landmarks directly in a list
                elif len(data) > 0 and isinstance(data[0], list) and len(data[0]) == 21:
                    coords = data[0]  # first hand's landmarks from a list of hand landmarks
                elif len(data) == 21 and isinstance(data[0], dict):
                    coords = data  # list of 21 dicts (each with x,y maybe)
            elif isinstance(data, dict):
                # If data is a dict, look for a key that contains landmark list
                for key in ['landmarks', 'hand_landmarks', 'points', 'coordinates']:
                    if key in data and isinstance(data[key], list):
                        val = data[key]
                        if len(val) == 21:
                            coords = val  # found 21 landmarks in the list
                        elif len(val) > 0 and isinstance(val[0], list) and len(val[0]) == 21:
                            coords = val[0]  # first set of 21 landmarks
                        elif len(val) == 21 and isinstance(val[0], dict):
                            coords = val  # list of 21 coordinate dicts
                        # If found the right key, no need to check other keys
                        if coords is not None:
                            break

    # Determine if this image has a full hand detection
    full_hand_detected = False

    if coords is not None:
        # If we have coordinates loaded, check if they represent 21 landmarks
        if isinstance(coords, list) and len(coords) == 21:
            full_hand_detected = True
    else:
        # No coordinate data available; fall back to analyzing the image for green landmarks.
        image = cv2.imread(file_path)
        if image is not None:
            # Define the color range for the green landmark dots (in BGR color space)
            lower_green = np.array([0, 250, 0], dtype=np.uint8)  # allow slight variation in green
            upper_green = np.array([0, 255, 0], dtype=np.uint8)
            mask = cv2.inRange(image, lower_green, upper_green)
            # Check if there are any green pixels in the mask
            if np.count_nonzero(mask) > 0:
                full_hand_detected = True

    # If a full hand (21 landmarks) is detected, add the filename to the valid list
    if full_hand_detected:
        valid_images.append(filename)

# Print the filenames of images with valid full-hand detections
# print("Images with full 21-landmark detections:")
# for fname in valid_images:
#     print(fname)

with open(os.path.join(project_dir, "valid_thermal_images.txt"), "w") as f:
    for fname in valid_images:
        f.write(fname + "\n")

In [ ]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
import os
import glob
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)

    # Dilate mask to preserve fingers and edges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)

    # Blur only the border by subtracting original from dilated
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)

    # Create smooth transition alpha mask
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)

    blended = (image * alpha).astype(np.uint8)
    return blended

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size[0], desired_size[1]
min_area = 1000

# === Load SAM ===
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "models", "sam_vit_h_4b8939.pth")
).to("cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

# === Loop through subfolders ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output6")
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n📁 Processing folder: {colormap_folder}")

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            print(f"\n🔄 Processing image: {filename}")

            image = cv2.imread(file_path)
            # Moved this line to save part
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if image.shape[1] != desired_size[0] or image.shape[0] != desired_size[1]:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            image = enhance_contrast(image)

            masks = mask_generator.generate(image)

            image_area = h * w
            valid_masks = []
            used_indexes = set()
            width_upper, width_lower, area_lower, area_higher = 0.5, 0.15, 0.1, 0.33

            # Filter masks based on area ratio
            for i, m in enumerate(masks):
                if i in used_indexes:
                    continue  # Skip already used masks
                mask = m["segmentation"].astype(np.uint8)
                area = cv2.countNonZero(mask)
                area_ratio = area / image_area

                # === Width check ===
                x, y, mask_w, mask_h = cv2.boundingRect(mask)
                width_ratio = mask_w / w
                height_ratio = mask_h / h
                if width_ratio > width_upper or width_ratio < width_lower or height_ratio > 0.8:
                    continue

                if area_lower <= area_ratio <= area_higher:
                    valid_masks.append((i, mask))
                    used_indexes.add(i)
                    if len(valid_masks) == 2:
                        break

            if len(valid_masks) < 2:
                print("↪️ Trying split-image fallback.")

                mid_x = w // 2
                left_half = image[:, :mid_x]
                right_half = image[:, mid_x:]

                valid_masks = []

                for half_image, offset_x, label in [(left_half, 0, "left"), (right_half, mid_x, "right")]:
                    half_masks = mask_generator.generate(half_image)
                    for i, m in enumerate(half_masks):
                        if i in used_indexes:
                            continue  # Skip already used masks
                        mask = m["segmentation"].astype(np.uint8)
                        area = cv2.countNonZero(mask)
                        area_ratio = area / (h * (w // 2))

                        x, y, mask_w, mask_h = cv2.boundingRect(mask)
                        width_ratio = mask_w / (w // 2)
                        height_ratio = mask_h / h

                        if not ((width_lower / 2) <= width_ratio <= width_upper and height_ratio <= 0.8):
                            continue
                        if 0.06 <= area_ratio <= 0.33:
                            # Shift mask to full image size
                            full_mask = np.zeros((h, w), dtype=np.uint8)
                            full_mask[:, offset_x:offset_x + (w // 2)] = mask
                            valid_masks.append((i, full_mask))
                            print(f"Found {label} hand in split image.")
                            break  # Only one hand needed per side

            # Save the two valid masks
            base_name = os.path.splitext(filename)[0]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            for i, (idx, mask) in enumerate(valid_masks):
                feathered_hand = feather_edges(image, mask, feather_amount=5)
                side = "left" if i == 0 else "right"
                output_path = os.path.join(output_dir, f"{base_name}_{side}_hand.jpg")
                cv2.imwrite(output_path, cv2.cvtColor(feathered_hand, cv2.COLOR_RGB2BGR))
                print(f"Saved {side} hand (mask {idx}) to: {output_path}")   

In [2]:
import os
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# === Load SAM ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "backend", "models", "sam_vit_h_4b8939.pth")
).to("cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

# === Load MediaPipe model ===
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options_two_hand = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=
                             os.path.join(project_dir, "backend", 
                                          "models", "hand_landmarker.task")),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

landmarker_two_hand = vision.HandLandmarker.create_from_options(options_two_hand)

options_one_hand = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=
                             os.path.join(project_dir, "backend", 
                                          "models", "hand_landmarker.task")),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=1
)

landmarker_one_hand = vision.HandLandmarker.create_from_options(options_one_hand)

In [ ]:
import os
import glob
import cv2
import numpy as np

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)

    # Dilate mask to preserve fingers and edges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)

    # Blur only the border by subtracting original from dilated
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)

    # Create smooth transition alpha mask
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)

    blended = (image * alpha).astype(np.uint8)
    return blended

def count_hand_landmarks(image):
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    result = landmarker.detect(mp_image)
    return sum(len(hand) for hand in result.hand_landmarks) if result.hand_landmarks else 0


def average_min_distance_to_corners(mask):
    h, w = mask.shape

    # === Check center color distance to undesired color ===
    center_pixel = mask[h // 2, w // 2].astype(np.float32)
    undesired_color = np.array([181, 192, 219], dtype=np.float32)

    # Euclidean color distance in RGB space
    color_dist = np.linalg.norm(center_pixel - undesired_color)

    # Normalize to [0, 1] range (max distance in RGB is sqrt(3 * 255^2))
    max_rgb_dist = np.sqrt(3 * 255**2)
    color_similarity_penalty = color_dist / max_rgb_dist  # 1 = far, 0 = exact match

    corners = np.array([
        [0, 0],              # top-left
        [0, w - 1],          # top-right
        [h // 2, 0],         # middle-left
        [h // 2, w - 1],     # middle-right
        [0, w // 2],         # top-middle
        [0, w // 2]          # count center twice
    ])

    ys, xs = np.nonzero(mask)
    points = np.stack([ys, xs], axis=1)

    if len(points) == 0:
        return np.inf  # empty mask

    # Base distance score
    min_dists = [np.min(np.linalg.norm(points - corner, axis=1)) for corner in corners]
    score = np.sum(min_dists)

    # === Center Proximity Score ===
    mask_center = np.mean(points, axis=0)  # (y, x)
    center = np.array([h // 2, w // 2])
    avg_center_dist = np.linalg.norm(mask_center - center)

    # Max possible distance = image diagonal (for normalization)
    max_dist = np.linalg.norm([h / 2, w / 2])
    center_score = (1 - (avg_center_dist / max_dist)) * score  # closer = higher score
    score += center_score

    # === Punish if mask touches any edge ===
    if np.any(mask[0, :]):            # top edge
        score *= 0.5
    if np.any(mask[0:h//2, 0]):            # left edge
        score *= 0.5
    if np.any(mask[0:h//2, w - 1]):        # right edge
        score *= 0.5

    # === Apply color similarity penalty ===
    score *= color_similarity_penalty  # closer to undesired color -> score is down-weighted

    # === Apply area size penalty
    mask_area_ratio = len(points) / (h * w)

    score *= np.log1p(mask_area_ratio * 100)  # log1p = log(1 + x)

    return abs(score)

# Filter masks based on area ratio
def filterMasks(width_upper, width_lower, area_lower, area_higher, original_image, masks):
    valid_masks, valid_dist = [], []
    for i, m in enumerate(masks):
        mask = m["segmentation"].astype(np.uint8)
        area = cv2.countNonZero(mask)
        area_ratio = area / image_area

        masked_image = feather_edges(original_image, mask, feather_amount=5)
        num_landmarks = count_hand_landmarks(masked_image)

        if (num_landmarks != 42):
            continue

        # === Width check ===
        x, y, mask_w, mask_h = cv2.boundingRect(mask)
        width_ratio = mask_w / w
        height_ratio = mask_h / h
        if width_ratio > width_upper or width_ratio < width_lower or height_ratio > 0.9:
            continue
        
        if area_lower <= area_ratio <= area_higher:
            valid_masks.append((i, mask))
            valid_dist.append(average_min_distance_to_corners(mask))
    return valid_masks, valid_dist

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size[0], desired_size[1]
min_area = 1000

# === Loop through subfolders ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output_white_punished")
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n📁 Processing folder: {colormap_folder}")

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            print(f"\n🔄 Processing image: {filename}")
            image = cv2.imread(file_path)

            if image.shape[1] != desired_size[0] or image.shape[0] != desired_size[1]:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            #image = enhance_contrast(image)

            masks = mask_generator.generate(image)

            image_area = h * w
            valid_masks = []
            valid_dist = []
            width_upper, width_lower, area_lower, area_higher = 0.95, 0.1, 0.08, 0.82
            valid_masks, valid_dist = filterMasks(width_upper,
                                                  width_lower,
                                                  area_lower,
                                                  area_higher,
                                                  image,
                                                  masks)
            if not valid_masks:
                print("couldn't find masks, trying with lower width limit again")
                valid_masks, valid_dist = filterMasks(width_upper + 0.05, width_lower - 0.05,
                            area_lower - 0.03, area_higher, image, masks)
            save_mask = []
            max_dist = -1
            best_index = -1
            for idx in range(len(valid_dist)):
                if valid_dist[idx] > max_dist:
                    max_dist = valid_dist[idx]
                    best_index = idx
            if best_index != -1:
                save_mask.append(valid_masks[best_index])
            # Save the two valid masks
            base_name = os.path.splitext(filename)[0]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            for i, (idx, mask) in enumerate(save_mask):
                feathered_hand = feather_edges(image, mask, feather_amount=5)
                output_path = os.path.join(output_dir, f"hands_{base_name}.jpg")
                cv2.imwrite(output_path, cv2.cvtColor(feathered_hand, cv2.COLOR_BGR2RGB))
                print(f"Saved hands (mask {idx}) to: {output_path}")   

Best optical segmentation!

In [ ]:
import os
import glob
import cv2
import numpy as np

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)

    # Dilate mask to preserve fingers and edges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)

    # Blur only the border by subtracting original from dilated
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)

    # Create smooth transition alpha mask
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)

    blended = (image * alpha).astype(np.uint8)
    return blended

def count_hand_landmarks(landmarker, image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    result = landmarker.detect(mp_image)
    return sum(len(hand) for hand in result.hand_landmarks) if result.hand_landmarks else 0

def average_min_distance_to_corners(mask):
    h, w = mask.shape

    corners = np.array([
        [0, 0],              # top-left
        [0, w - 1],          # top-right
        [h // 2, 0],         # middle-left
        [h // 2, w - 1],     # middle-right
        [0, w // 2],         # top-middle
        [0, w // 2]          # count center twice
    ])

    ys, xs = np.nonzero(mask)
    points = np.stack([ys, xs], axis=1)

    if len(points) == 0:
        return np.inf  # empty mask

    # Base distance score
    min_dists = [np.min(np.linalg.norm(points - corner, axis=1)) for corner in corners]
    score = np.sum(min_dists)

    # === Center Proximity Score ===
    mask_center = np.mean(points, axis=0)  # (y, x)
    center = np.array([h // 2, w // 2])
    avg_center_dist = np.linalg.norm(mask_center - center)

    # Max possible distance = image diagonal (for normalization)
    max_dist = np.linalg.norm([h / 2, w / 2])
    center_score = (1 + avg_center_dist / max_dist) * score / 2
    score += center_score

    # === Punish if mask touches any edge ===
    if np.any(mask[0, :]):            # top edge
        score *= 0.5
    if np.any(mask[0:h//2, 0]):            # left edge
        score *= 0.5
    if np.any(mask[0:h//2, w - 1]):        # right edge
        score *= 0.5

    return abs(score)

# Filter masks based on area ratio
def filterMasks(width_upper, width_lower, area_lower, area_higher, original_image, masks, landmarker, num_hands):
    valid_masks, valid_dist = [], []
    for i, m in enumerate(masks):
        mask = m["segmentation"].astype(np.uint8)
        area = cv2.countNonZero(mask)
        area_ratio = area / image_area

        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        masked_image = feather_edges(original_image, mask, feather_amount=5)
        num_landmarks = count_hand_landmarks(landmarker, masked_image)

        if (num_hands == 2 and num_landmarks != 42):
            continue

        if (num_hands == 1 and num_landmarks != 21):
            continue

        # === Width check ===
        x, y, mask_w, mask_h = cv2.boundingRect(mask)
        width_ratio = mask_w / w
        height_ratio = mask_h / h
        # if width_ratio > width_upper or width_ratio < width_lower or height_ratio > 0.9:
        #     continue
        
        if area_lower <= area_ratio <= area_higher:
            valid_masks.append((i, mask))
            valid_dist.append(average_min_distance_to_corners(mask))
    return valid_masks, valid_dist

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size[0], desired_size[1]
min_area = 1000

# === Loop through subfolders ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output_landmark_combined")
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n📁 Processing folder: {colormap_folder}")

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            print(f"\n🔄 Processing image: {filename}")
            image = cv2.imread(file_path)

            if image.shape[1] != desired_size[0] or image.shape[0] != desired_size[1]:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            #image = enhance_contrast(image)

            masks = mask_generator.generate(image)

            image_area = h * w
            valid_masks = []
            valid_dist = []
            two_separate = False
            width_upper, width_lower, area_lower, area_higher = 0.9, 0.2, 0.06, 0.82
            valid_masks, valid_dist = filterMasks(width_upper,
                                                  width_lower,
                                                  area_lower,
                                                  area_higher,
                                                  image,
                                                  masks,
                                                  landmarker_two_hand,
                                                  2)
            if not valid_masks:
                print("couldn't find masks, trying to find one hand at a time")
                valid_masks, valid_dist = filterMasks(width_upper + 0.05, 
                                                      width_lower - 0.05,
                                                      area_lower - 0.03, 
                                                      area_higher + 0.08,
                                                      image, 
                                                      masks, 
                                                      landmarker_one_hand,
                                                      1)
                two_separate = True
            save_mask = []
            if ( two_separate ):
                scored_masks = sorted(zip(valid_dist, valid_masks), key=lambda x: x[0], reverse=True)
                
                if not scored_masks:
                    continue
                elif len(scored_masks) > 1:
                    top_masks = scored_masks[:2]
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    feathered_hands = ["feathered_hand_1", "feathered_hand_2"]
                    for i, (score, (idx, mask)) in enumerate(top_masks):
                        feathered_hands[i] = feather_edges(image, mask, feather_amount=5)
                    combined = cv2.add(feathered_hands[0], feathered_hands[1])
                    base_name = os.path.splitext(filename)[0]
                    output_path = os.path.join(output_dir, f"hands_{base_name}.jpg")
                    # Check for 42 landmarks before saving
                    num_landmarks = count_hand_landmarks(landmarker_two_hand, combined)
                    if num_landmarks == 42:
                        cv2.imwrite(output_path, cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
                        print(f"Saved hands (mask combined) to: {output_path}")
                    else:
                        print("Skipped: Landmark count != 42 after combining masks.")

            else:
                max_dist = -1
                best_index = -1
                for idx in range(len(valid_dist)):
                    if valid_dist[idx] > max_dist:
                        max_dist = valid_dist[idx]
                        best_index = idx
                if best_index != -1:
                    save_mask.append(valid_masks[best_index])
                # Save the two valid masks
                base_name = os.path.splitext(filename)[0]
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                for i, (idx, mask) in enumerate(save_mask):
                    feathered_hand = feather_edges(image, mask, feather_amount=5)
                    output_path = os.path.join(output_dir, f"hands_{base_name}.jpg")
                    num_landmarks = count_hand_landmarks(landmarker_two_hand, feathered_hand)
                    if num_landmarks == 42:
                        cv2.imwrite(output_path, cv2.cvtColor(feathered_hand, cv2.COLOR_BGR2RGB))
                        print(f"Saved hands (mask {idx}) to: {output_path}")
                    else:
                        print("Skipped: Landmark count != 42 on selected mask.")

In [ ]:
import os
import glob
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)
    blended = (image * alpha).astype(np.uint8)
    return blended

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size

# === Load SAM ===
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "models", "sam_vit_h_4b8939.pth")
).to("cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

# === Processing ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output_final")
        os.makedirs(output_dir, exist_ok=True)

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            image = cv2.imread(file_path)

            if image.shape[1] != w or image.shape[0] != h:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            enhanced_image = enhance_contrast(image_rgb)

            masks = mask_generator.generate(enhanced_image)

            valid_masks = []
            for m in masks:
                mask = m["segmentation"].astype(np.uint8)
                area_ratio = cv2.countNonZero(mask) / (h * w)

                x, y, mask_w, mask_h = cv2.boundingRect(mask)
                width_ratio = mask_w / w
                height_ratio = mask_h / h
                if 0.2 <= width_ratio <= 0.9 and height_ratio <= 0.9 and 0.15 <= area_ratio <= 0.8:
                    valid_masks.append(mask)

            image_center_x = w / 2
            base_name = os.path.splitext(filename)[0]

            for idx, mask in enumerate(valid_masks):
                x, y, mask_w, mask_h = cv2.boundingRect(mask)
                mask_center_x = x + mask_w / 2
                side = "left" if mask_center_x < image_center_x else "right"

                # Save feathered hand
                feathered_hand = feather_edges(enhanced_image, mask, feather_amount=5)
                output_hand_path = os.path.join(output_dir, f"{base_name}_{side}_hand.jpg")
                cv2.imwrite(output_hand_path, cv2.cvtColor(feathered_hand, cv2.COLOR_RGB2BGR))

                # Save binary mask
                binary_mask_output = (mask * 255).astype(np.uint8)
                output_mask_path = os.path.join(output_dir, f"{base_name}_{side}_mask.png")
                cv2.imwrite(output_mask_path, binary_mask_output)

                print(f"Saved {side} hand and mask to: {output_hand_path}, {output_mask_path}")

In [ ]:
import os
import glob
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)
    blended = (image * alpha).astype(np.uint8)
    return blended

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size

# === Load SAM ===
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "models", "sam_vit_h_4b8939.pth")
).to("cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

# === Processing ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output_final2")
        os.makedirs(output_dir, exist_ok=True)

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            image = cv2.imread(file_path)

            if image.shape[1] != w or image.shape[0] != h:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            enhanced_image = enhance_contrast(image_rgb)

            masks = mask_generator.generate(enhanced_image)

            combined_mask = np.zeros((h, w), dtype=np.uint8)
            for m in masks:
                mask = m["segmentation"].astype(np.uint8)
                area_ratio = cv2.countNonZero(mask) / (h * w)

                x, y, mask_w, mask_h = cv2.boundingRect(mask)
                width_ratio = mask_w / w
                height_ratio = mask_h / h
                if 0.2 <= width_ratio <= 0.9 and height_ratio <= 0.9 and 0.15 <= area_ratio <= 0.8:
                    combined_mask = cv2.bitwise_or(combined_mask, mask)

            feathered_hands = feather_edges(enhanced_image, combined_mask, feather_amount=5)
            output_hand_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_hands.jpg")
            cv2.imwrite(output_hand_path, cv2.cvtColor(feathered_hands, cv2.COLOR_RGB2BGR))

            binary_mask_output = (combined_mask * 255).astype(np.uint8)
            output_mask_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_mask.png")
            cv2.imwrite(output_mask_path, binary_mask_output)

            print(f"Saved combined hands and mask to: {output_hand_path}, {output_mask_path}")

In [ ]:
import os
import glob
import cv2
import numpy as np
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

# === Enhance contrast ===
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

# === Improved feathering ===
def feather_edges(image, mask, feather_amount=5):
    mask = mask.astype(np.float32)

    # Dilate mask to preserve fingers and edges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)

    # Blur only the border by subtracting original from dilated
    edge = mask_dilated - mask
    edge_blur = cv2.GaussianBlur(edge, (0, 0), feather_amount)

    # Create smooth transition alpha mask
    soft_mask = np.clip(mask + edge_blur, 0, 1)
    alpha = np.stack([soft_mask] * 3, axis=-1)

    blended = (image * alpha).astype(np.uint8)
    return blended

def min_distance_to_corners(mask):
    h, w = mask.shape
    corners = np.array([
        [0, 0],          # top-left
        [0, w - 1],      # top-right
        [h - 1, 0],      # bottom-left
        [h - 1, w - 1]   # bottom-right
    ])

    # Get all non-zero (y, x) points in the mask
    ys, xs = np.nonzero(mask)
    points = np.stack([ys, xs], axis=1)

    if len(points) == 0:
        return np.inf  # if the mask is empty

    # Compute distances from each point to each corner
    dists = np.linalg.norm(points[:, None, :] - corners[None, :, :], axis=2)
    dist = dists.min()  # closest any pixel gets to a corner

    return dist

# === Settings ===
project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
images_base_dir = os.path.join(project_dir, "data", "images")
desired_size = (320, 240)
w, h = desired_size[0], desired_size[1]
min_area = 1000

# === Load SAM ===
sam = sam_model_registry["vit_h"](
    checkpoint=os.path.join(project_dir, "models", "sam_vit_h_4b8939.pth")
).to("cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

# === Loop through subfolders ===
for colormap_folder in os.listdir(images_base_dir):
    if colormap_folder == "optical":
        folder_path = os.path.join(images_base_dir, colormap_folder)
        if not os.path.isdir(folder_path):
            continue

        output_dir = os.path.join(project_dir, f"{colormap_folder}_output7")
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n📁 Processing folder: {colormap_folder}")

        for file_path in glob.glob(f"{folder_path}/*.jpg"):
            filename = os.path.basename(file_path)
            print(f"\n🔄 Processing image: {filename}")

            image = cv2.imread(file_path)
            # Moved this line to save part
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if image.shape[1] != desired_size[0] or image.shape[0] != desired_size[1]:
                image = cv2.resize(image, desired_size, interpolation=cv2.INTER_AREA)

            image = enhance_contrast(image)

            masks = mask_generator.generate(image)

            masks = masks[:2]
            image_area = h * w
            valid_masks = []
            width_upper, width_lower, area_lower, area_higher = 0.9, 0.4, 0.2, 0.5

            # mask1 and mask2 are binary masks (np.uint8) of the same shape
            dist1 = min_distance_to_corners(masks[0]["segmentation"].astype(np.uint8))
            dist2 = min_distance_to_corners(masks[1]["segmentation"].astype(np.uint8))
            dist3 = min_distance_to_corners(masks[2]["segmentation"].astype(np.uint8))

            if dist2 < dist3 and dist1 < dist3:
                print("Mask 2 is furthest from a corner")
                valid_masks.append((2, masks[2]["segmentation"].astype(np.uint8)))
            elif dist1 < dist2 and dist3 < dist2:
                print("Mask 1 is furthest from a corner")
                valid_masks.append((1, masks[1]["segmentation"].astype(np.uint8)))
            else:
                print("Mask 0 is furthest from a corner")
                valid_masks.append((0, masks[0]["segmentation"].astype(np.uint8)))
            # Filter masks based on area ratio
            #for i, m in enumerate(masks):
            #    mask = m["segmentation"].astype(np.uint8)
            #    area = cv2.countNonZero(mask)
                #area_ratio = area / image_area

                # === Width check ===
                #x, y, mask_w, mask_h = cv2.boundingRect(mask)
                #width_ratio = mask_w / w
                #height_ratio = mask_h / h
                #if width_ratio > width_upper or width_ratio < width_lower or height_ratio > 0.8:
                    #continue

                #if area_lower <= area_ratio <= area_higher:
                # mask: binary mask of shape (height, width)
            #    h, w = mask.shape

                # Corners as (row, col) = (y, x)
            #    corners = [
            #        (0, 0), (0, w - 1),(h - 1, 0),(h - 1, w - 1)]

                # Check if any corner is part of the mask
                #corner_present = any(mask[y, x] > 0 for y, x in corners)

                #if corner_present:
                #    print("Mask touches a corner – might be background or invalid.")
                #else:
                #    valid_masks.append((i, mask))
                    #break

            # Save the two valid masks
            base_name = os.path.splitext(filename)[0]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            for i, (idx, mask) in enumerate(valid_masks):
                feathered_hand = feather_edges(image, mask, feather_amount=5)
                #side = "left" if i == 0 else "right"
                output_path = os.path.join(output_dir, f"{base_name}_hand.jpg")
                cv2.imwrite(output_path, cv2.cvtColor(feathered_hand, cv2.COLOR_RGB2BGR))
                print(f"✅ Saved hands (mask {idx}) to: {output_path}")

In [ ]:
import os
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

project_dir = "/Users/Bunni/Documents/FinalProject/ThermoML"
input_dir = os.path.join(project_dir, "gnuplot2_output")
output_dir = os.path.join(project_dir, "gnuplot2_mediapipe_landmarks")
os.makedirs(output_dir, exist_ok=True)

model_path = os.path.join(project_dir, "models", "hand_landmarker.task")
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = vision.HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2
)

landmarker = vision.HandLandmarker.create_from_options(options)

for file in os.listdir(input_dir):
    if not file.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    image_path = os.path.join(input_dir, file)
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

    result = landmarker.detect(mp_image)

    if result.hand_landmarks:
        for hand in result.hand_landmarks:
            for landmark in hand:
                x_px = int(landmark.x * image.shape[1])
                y_px = int(landmark.y * image.shape[0])
                cv2.circle(image, (x_px, y_px), 3, (0, 255, 0), -1)

    output_path = os.path.join(output_dir, f"landmarks_{file}")
    cv2.imwrite(output_path, image)
    print(f"Saved landmark image to: {output_path}")